Способы ввода значений:

1. Ручной ввод
2. Рандомные числа
3. Из файла csv

In [2]:
import numpy as np
import pandas as pd
import sys, csv
from itertools import islice
from scipy.optimize import linprog
from matplotlib import pyplot as plt
import random
from tkinter import Frame, Tk, BOTH, Text, Menu, END
from tkinter import filedialog


way = input('Каким способом Вы хотите ввести значения? ') # Пользователь выбирает способ ввода данных
if way == '1': # Ручной ввод данных
    
    # Вводим количество строк и столбцов матрицы исходных значений
    size_rows = int(input("Количество строк (пунктов отправления): ")) 
    size_cols = int(input("Количество столбцов (пунктов назначения): "))
    
    # Цикл для последовательного ввода значений и заполнения ими матрицы исходных данных
    matrix = []
    for i in range(size_rows):
        rows = []
        for j in range(size_cols):
            x = input('Введите значение: ')
            rows.append(float(x))
        matrix.append(rows)
    
    # Вводим потребности и производительность
    max_company_1 = input('Потребности (ввод через запятую): ')
    max_factory_2 = input('Производительность (ввод через запятую): ')

    # Разбиваем строку с введенными потребностями на список
    max_company, max_factory = [],[]
    max_company_1_2 = max_company_1.split(',')
    # Переводим значения в полученном списке в тип int
    for i in max_company_1_2:
        max_company.append(int(i))

    # Разбиваем строку с введенными значениями производительности на список
    max_factory_2_2 = max_factory_2.split(',')
    # Переводим значения в полученном списке в тип int
    for i in max_factory_2_2:
        max_factory.append(int(i))

elif way == '2': # Заполнение данных случайными значениями
    # Вводим количество строк и столбцов матрицы исходных значений
    size_rows = int(input("Количество строк (пунктов отправления): ")) 
    size_cols = int(input("Количество столбцов (пунктов назначения): "))

    # Последовательно генерируем случайные числа типа int в диапазоне (0;1000) и добавляем их в матрицу
    matrix = []
    for i in range(size_rows):
        rows = []
        for j in range(size_cols):
            x = random.randint(0,1000)
            rows.append(float(x))
        matrix.append(rows)
    
    # Выводим на экран получившуюся матрицу
    print('Входные значения: \n', pd.DataFrame(np.array(matrix)))
    
    # Заполняем список с потребностями случайными числами типа int
    max_company = []
    for i in range(size_cols):
        max_company.append(random.randint(0,1000))
    print('Потребности: ', max_company) # Выводим на экран списко потребностей (для каждого столбца матрицы)
    
    # Заполняем список с производительностью случайными числами типа int
    error = 0
    while error != 1:
        max_factory = []
        for i in range(size_rows):
            max_factory.append(random.randint(0, sum(max_company)))
        if sum(max_factory) >= sum(max_company):
            error = 1
    print('Производительность: ', max_factory) # Выводим на экран список производительности (для каждой строки матрицы)

    
elif way == '3': # Вывод данных из файла csv
    
    # Класс для выполнения всех операций с tkinter
    class Example(Frame):
        # функция, которая обращается к классу, от которого наследуется текущий
        def __init__(self):
            super().__init__()
            self.initUI()
        
        # Функция устанавливает раздел "файл" в окне для возможности выбора самого файла
        def initUI(self):
            self.master.title("Окно для выбора файла")
            self.pack(fill=BOTH, expand=1)

            menubar = Menu(self.master)
            self.master.config(menu=menubar)

            fileMenu = Menu(menubar)
            fileMenu.add_command(label="Открыть", command=self.onOpen)
            menubar.add_cascade(label="Файл", menu=fileMenu)

            self.txt = Text(self)
            self.txt.pack(fill=BOTH, expand=1)
        
        # Функция дает возможность выбора формата файла (csv или "все файлы")
        def onOpen(self):
            ftypes = [('Файлы csv', '*.csv'), ('Все файлы', '*')]
            dlg = filedialog.Open(self, filetypes = ftypes)
            fl = dlg.show()

            if fl != '':
                text = self.readFile(fl)
                self.txt.insert(END, text)
        
        # Функция открывает файл, считывает данные с него и затем выполняет все нужные операции для поиска оптимального решения
        def readFile(self, filename):
            results = []

            with open(filename) as f:
                row = [s for line in f.readlines() for s in line[:-1].split(',')]
            print(row)
            
            # Для отсчитывания количества столбцов в будущей матрице значений
            k = -1
            rows = []
            for i in row:
                k += 1
                if i == 'Производительность':
                    break
            
            # Преобразуем потребности в список
            max_company_1_2 = list(islice(reversed(row), 0, k))
            max_company_1_2.reverse()
            
            # Цикл для преобразования производительности в список
            max_factory = []
            matrix = []
            num = k+1
            for i in range(int((len(row)-k-k+1)/k)-1):
                for j in range(k):
                    rows.append(row[num])
                    num += 1
                max_factory.append(int(row[(num)]))
                matrix.append(rows)
                num += 1
                rows = []
                
                max_company = []
                # Потребности
                for i in max_company_1_2:
                    max_company.append(int(i))
            
            # Вывод всх исходных данных из файла на экран
            print('Входные значения: \n', pd.DataFrame(np.array(matrix)))
            print('Потребности: ', max_company)
            print('Производительность ', max_factory)
            
            size_cols = k # Количество столбцов в матрице значений
            k += 1
            size_rows = int((len(row)-k-k+1)/k) # # Количество строк в матрице значений
            
            
            # С этого момента комментарии те же самые, что и к фрагменту кода, начиная со строки 271
            print('\n')
            # Целевая функция
            c = sum(matrix, [])
            function_text = ''
            for i in range(1,len(c)+1):
                function_text += 'x' + str(i) + '*' + str(c[i-1]) + ' + '
            function_text = function_text[:len(function_text)-2]
            print(function_text)

            # Создаем список для A_ub (для ограничений в функции linprog)
            sp_Aub_2 = []
            sp_Aub = []
            k1 = 0
            for i in range(size_rows):
                for j in range(size_rows):
                    n = 1
                    while n <= size_cols:
                        if j == k1:
                            sp_Aub_2.append(1)  
                        else:
                            sp_Aub_2.append(0)
                        n += 1
                k1 += 1
                sp_Aub.append(sp_Aub_2)
                sp_Aub_2 = []

            # Создаем список для A_eq
            sp_Aeq_2 = []
            sp_Aeq = []
            k1 = 0
            for i in range(size_cols):
                for j in range(size_rows):
                    for k in range(size_cols):
                        if k == i:
                            sp_Aeq_2.append(1)
                            k += 1
                        else:
                            sp_Aeq_2.append(0)
                    k -= 1
                sp_Aeq.append(sp_Aeq_2)
                sp_Aeq_2 = []

            b_ub = max_factory
            A_ub = sp_Aub
            b_eq = max_company
            A_eq = sp_Aeq


            # Считаем целевую функцию и округляем ее значение до 3-х знаков
            function = round(linprog(c, A_ub, b_ub, A_eq, b_eq).fun, 3)
            print('Целевая функция', function)
            array1 = linprog(c, A_ub, b_ub, A_eq, b_eq).x
            array0 = np.array([round(i, 1) for i in array1]) # Округляем искомые значения до 1 знака

            # Преобразуем список полученных значений в матрицу
            array = []
            array_sm = []
            l = 0
            for i in range(size_rows):
                for j in range(size_cols):
                    array_sm.append(array0[l])
                    l += 1
                array.append(array_sm)
                array_sm = []
            array = np.array(array)
            print(pd.DataFrame(array))

            if (size_cols == 2 and size_rows == 1) or (size_cols == 1 and size_rows == 2):
                # В зависимости от размера матрицы (2x1 или 1x2) присваиваем переменным значения ограничений для последующего построения графиков функций
                if size_cols == 2 and size_rows == 1:
                    q,h = max_factory, max_company
                elif size_cols == 1 and size_rows == 2:
                    q,h = max_company, max_factory

                # Задаем зависимость переменных: 
                x1 = np.linspace(0, q[0], 50)
                
                x2 = [q[0] - i for i in x1]    # Ограничение x2 = a - x1
                x3 = [h[1] for i in x1]        # Ограничение x1 = b2
                x4 = [0 for i in x1]           # Ограничение x1 >= 0
                plt.axvline(x=h[0], label = 'x1 <= b1', color = 'g')  # Ограничение x1 = b1
                plt.axvline(x=0, label = 'x2 >= 0', color = 'y')               # Ограничение x2 >= 0

                # Линия уровня
                # При l = 0   Линия уровня: c1*x1 + c2*x2 = l
                l = [(function - int(c[0])*i) / int(c[1]) for i in x1]

                # Построение графика
                plt.plot(x1, x2, label = 'x2 <= a - x1', color = 'r')
                plt.plot(x1, x3, label = 'x2 <= b2', color = 'b')
                plt.plot(x1, x4, label = 'x1 >= 0', color = 'c')
                plt.plot(x1, l, label = 'Линия уровня', color = 'k', linestyle = '--')

                # Заливка
                if size_cols == 2 and size_rows == 1:
                    x = [0, h[0], h[0], 0]
                    y = [0, 0, h[1], h[1]]

                elif size_cols == 1 and size_rows == 2:
                    x = [0, h[0], h[0], q[0] - h[1], 0]
                    y = [0, 0, q[0] - h[0], h[1], h[1]]
                plt.fill(x, y, color='red', alpha=0.3)    


                plt.title('Визуализация')       # название графика
                plt.xlabel('x1')                # ось абсцисс
                plt.ylabel('x2')                # ось ординат
                plt.grid()                      # включение отображения сетки
                
                plt.legend(fontsize = 12, ncol = 2, title = 'Ограничения', title_fontsize = '12', bbox_to_anchor=(1, 1))
            else:
                print('Визуализация недоступна. Введите матрицу 2x1 или 1x2')

            return row
    
    # Функция отображает окно tkinter
    def main():
        root = Tk()
        ex = Example()
        root.geometry("300x250+300+300")
        root.mainloop()

    # Функция вызывает окно tkinter
    if __name__ == '__main__':
        main()

# Вывод пустой строчки для красоты
print('\n')

# Преобразование матрицы с исходными значениями (те, что вводили) в одномерный список для дальнейшего использования в функции linprog
# 'c' в функции linprog - это коэффициенты линейной целевой функции, которые необходимо минимизировать
c = sum(matrix, [])

# Формирование уравнения целевой функции в общем виде в форме строки
function_text = ''
for i in range(1,len(c)+1):
    function_text += 'x' + str(i) + '*' + str(c[i-1]) + ' + '
function_text = function_text[:len(function_text)-2]
print(function_text)

# Создаем двумерный список из ограничений на производительность (для ограничений в функции linprog)
# 'A_ub' в функции linprog - это матрица ограничений неравенства. Каждая строка A_ub задает коэффициенты ограничения линейного неравенства на x
sp_Aub_2 = []
sp_Aub = []
k1 = 0
for i in range(size_rows):
    for j in range(size_rows):
        n = 1
        while n <= size_cols:
            if j == k1:
                sp_Aub_2.append(1)  
            else:
                sp_Aub_2.append(0)
            n += 1
    k1 += 1
    sp_Aub.append(sp_Aub_2)
    sp_Aub_2 = []

# Создаем двумерный список из ограничений на потребности (для ограничений в функции linprog)
# 'A_eq' в функции linprog - это матрица ограничений равенства. Каждая строка A_eq задает коэффициенты ограничения линейного равенства на x.
sp_Aeq_2 = []
sp_Aeq = []
k1 = 0
for i in range(size_cols):
    for j in range(size_rows):
        for k in range(size_cols):
            if k == i:
                sp_Aeq_2.append(1)
                k += 1
            else:
                sp_Aeq_2.append(0)
        k -= 1
    sp_Aeq.append(sp_Aeq_2)
    sp_Aeq_2 = []

b_ub = max_factory # (Это для функции linprog) 'b_ub' в функции linprog - это вектор ограничения неравенства. Каждый элемент представляет собой верхнюю границу соответствующего значения A_ub
A_ub = sp_Aub
b_eq = max_company # (Это для функции linprog) 'b_eq' в функции linprog - это вектор ограничения равенства. Каждый элемент A_eq должен быть равен соответствующему элементу b_eq
A_eq = sp_Aeq


# Вбиваем в функцию linprog переменные, она находит оптимальное значение целевой функции, и округляем ее значение до 3-х знаков)
function = round(linprog(c, A_ub, b_ub, A_eq, b_eq).fun, 3)
print('Целевая функция', function) # Выводим на экран значение ЦелФун
array1 = linprog(c, A_ub, b_ub, A_eq, b_eq).x # Присваиваем переменной найденные оптимальные значения матрицы, полученной в виде списка
array0 = np.array([round(i, 1) for i in array1]) # И округляем все значения этого списка до 1 знака

# Преобразуем список полученных значений в матрицу
array = []
array_sm = []
l = 0
for i in range(size_rows):
    for j in range(size_cols):
        array_sm.append(array0[l])
        l += 1
    array.append(array_sm)
    array_sm = []
# Выводим на экран матрицу оптимальных значений, используя библиотеку pandas
print(pd.DataFrame(array))


if (size_cols == 2 and size_rows == 1) or (size_cols == 1 and size_rows == 2): # Если матрица размером 2x1 или 1x2, то делаем визуализацию
    
    # В зависимости от размера матрицы (2x1 или 1x2) присваиваем переменным значения ограничений для последующего построения графиков функций
    if size_cols == 2 and size_rows == 1:
        q,h = max_factory, max_company
    elif size_cols == 1 and size_rows == 2:
        q,h = max_company, max_factory
        
                
    # Задаем зависимость переменных: 
    x1 = np.linspace(0, q[0], 50)
    x2 = [q[0] - i for i in x1]    # Ограничение x2 = a - x1
    x3 = [h[1] for i in x1]        # Ограничение x1 = b2
    x4 = [0 for i in x1]           # Ограничение x1 >= 0
    plt.axvline(x=h[0], label = 'x1 <= b1', color = 'g')  # Ограничение x1 = b1
    plt.axvline(x=0, label = 'x2 >= 0', color = 'y')               # Ограничение x2 >= 0
        
    # Линия уровня
    # При l = 0   Линия уровня: c1*x1 + c2*x2 = l
    l = [(function - c[0]*i)/c[1] for i in x1]
    
    # Построение графика
    plt.plot(x1, x2, label = 'x2 <= a - x1', color = 'r')
    plt.plot(x1, x3, label = 'x2 <= b2', color = 'b')
    plt.plot(x1, x4, label = 'x1 >= 0', color = 'c')
    plt.plot(x1, l, label = 'Линия уровня', color = 'k', linestyle = '--')
    
    # Заливка
    if size_cols == 2 and size_rows == 1:
        x = [0, h[0], h[0], 0]
        y = [0, 0, h[1], h[1]]
        
    elif size_cols == 1 and size_rows == 2:
        x = [0, h[0], h[0], q[0] - h[1], 0]
        y = [0, 0, q[0] - h[0], h[1], h[1]]
    plt.fill(x, y, color='red', alpha=0.3)   
    
    plt.title('Визуализация')       # название графика
    plt.xlabel('x1')                # ось абсцисс
    plt.ylabel('x2')                # ось ординат
    plt.grid()                      # включение отображения сетки
    
    plt.legend(fontsize = 12,            #  размер шрифта
               ncol = 2,                 #  количество столбцов
               title = 'Ограничения',    #  заголовок
               title_fontsize = '12',    #  размер шрифта заголовка
               bbox_to_anchor=(1, 1))    #  размер шрифта заголовка
else:
    print('Визуализация недоступна. Введите матрицу 2x1 или 1x2') # Если матрица отлична от размеров 2x1 или 1x2, то выдаем это сообщение

Каким способом Вы хотите ввести значения? 1
Количество строк (пунктов отправления): 2
Количество столбцов (пунктов назначения): 8
Введите значение: 32
Введите значение: 15
Введите значение: 16
Введите значение: 12
Введите значение: 3
Введите значение: 5
Введите значение: 18
Введите значение: 22
Введите значение: 12
Введите значение: 4
Введите значение: 7
Введите значение: 16
Введите значение: 17
Введите значение: 23
Введите значение: 14
Введите значение: 5
Потребности (ввод через запятую): 34,25,65,42,65,12,44,25
Производительность (ввод через запятую): 150,200


x1*32.0 + x2*15.0 + x3*16.0 + x4*12.0 + x5*3.0 + x6*5.0 + x7*18.0 + x8*22.0 + x9*12.0 + x10*4.0 + x11*7.0 + x12*16.0 + x13*17.0 + x14*23.0 + x15*14.0 + x16*5.0 
Целевая функция 2463.0
      0     1     2     3     4     5     6     7
0   0.0   0.0   0.0  42.0  65.0  12.0   0.0   0.0
1  34.0  25.0  65.0   0.0   0.0   0.0  44.0  25.0
Визуализация недоступна. Введите матрицу 2x1 или 1x2


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
